In [ ]:
# import torch
import numpy as np
import os
import pandas as pd
import json

In [ ]:
# load the json file into a list
with open('./acsa-restaurant-large/acsa_train.json','rb') as f:
    data = json.load(f)

sentence_data = [x['sentence'] for x in data]
aspect_data = [x['aspect'] for x in data]
sentiment_data = [x['sentiment'] for x in data]

# print('Number of sentences: ', len(sentence_data))
# print('Number of aspects: ', len(aspect_data))
# print('Number of sentiments: ', len(sentiment_data))


In [ ]:
data_words = {}
for example in sentence_data:
    for word in example.split():
        if word[-1] in ['.',',','!','?']:
            word = word[:-1]
            if word not in data_words:
                data_words[word] = 0
            else:
                data_words[word] += 1

In [ ]:
# Load glove vectors
glove_folder = os.path.join(os.getcwd(), 'glove_file')

# get path of glove.6B.300d.txt file in test folder
glove_file = os.path.join(glove_folder, 'glove.6B.300d.txt')

def load_glove_vectors(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    with open(glove_file, 'r') as f:
        embs = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]

            if curr_word in data_words:
                try:
                    embedding = np.array([float(value) for value in line[1:]])
                    embs[curr_word] = embedding
                except:
                    print('error loading embedding')
    return words, word_to_vec_map, embs

glove_words, glove_word_to_vec_map, data_word_to_vec_map = load_glove_vectors(glove_file)
# print(len(words))
# print(len(word_to_vec_map))
# print(word_to_vec_map['the'])
# print(word_to_vec_map['the'].shape)


In [ ]:
# print(len(data_words))
# print(len(data_word_to_vec_map))
missing_words = len(data_words) - len(data_word_to_vec_map)
print(missing_words)

In [ ]:

word2indices = {}
indices2word = {}
for sentence in data:
    for word in sentence['sentence'].split():
        if word not in word2indices:
            word2indices[word] = len(word2indices)
            indices2word[word2indices[word]] = word
        
# print(word2indices)

# dictionary containing the word and its corresponding embedding
embs = {}
for word in word2indices:
    if word in word_to_vec_map:
        embs[word] = word_to_vec_map[word]
    else:
        embs[word] = np.random.randn(300)